<h3>Import libraries

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


<h3>Scrap data from webpage using requests and Beautiful soup

In [3]:
data=requests.get('https://en.wikipedia.org/wiki/Category:Neighbourhoods_in_Bangalore').text

In [4]:
soup= BeautifulSoup(data,'html.parser')

In [5]:
neighborhoodList = []

In [9]:
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
        neighborhoodList.append(row.text)
neighborhoodList=neighborhoodList[6:150]

In [10]:
banglore_df = pd.DataFrame({"Neighborhood": neighborhoodList})
banglore_df.head()

,Neighborhood
0,Banashankari
1,Banaswadi
2,Basavanagudi
3,Basaveshwaranagar
4,Bellandur


In [11]:
banglore_df.shape

(144, 1)

<h3>Get the Geographical locations

In [12]:
geolocator=Nominatim(user_agent='My-application')
def get_coordinates(neighborhood):
    latitude=None
    longitude=None
    neighborhood="/'"+neighborhood+"/'"
    location=geolocator.geocode(neighborhood)
    latitude=location.latitude
    longitude=location.longitude
    c=[latitude,longitude]
    return c

In [13]:
coords=[]
for neighborhood in banglore_df["Neighborhood"].tolist():
    try:
        coords.append(get_coordinates(neighborhood))
    except:
        coords.append(['n','n'])
        continue
coords

[[15.8876779, 75.7046777],
 [13.0141618, 77.6518539],
 [13.8345913, 75.7339359],
 [12.9937572, 77.5391325],
 [12.93577245, 77.66676103753434],
 [12.9996559, 76.0932984],
 [12.9785713, 77.4778584],
 [12.8995934, 77.610317336124],
 [12.9089453, 77.6239038],
 [12.8162443, 77.6916113],
 [33.5935063, -79.0345627],
 [12.9151772, 77.6102821],
 [13.062073550000001, 77.59639164365345],
 [12.9111998, 77.7065133],
 [12.9569966, 77.5633682],
 [17.4436387, 77.433391],
 [12.9680027, 77.578642],
 [12.9121916, 77.5476337],
 [13.0027852, 77.624747],
 [12.9954832, 77.6208684],
 [19.060047150000003, 72.92375205185107],
 [13.0994854, 77.8361008],
 [13.01582965, 77.60311172230354],
 [12.8877128, 77.6106443],
 [13.0420682, 77.56801529040317],
 [12.9624669, 77.6381958],
 [12.945245, 77.6269144],
 [-6.1938201, 106.8920008],
 ['n', 'n'],
 [12.9457463, 77.5706568],
 [12.9771534, 77.5784005],
 [13.0567639, 77.5467062],
 [13.024837, 77.592089],
 [12.993425, 77.7005156],
 [12.9401439, 77.5444995],
 [12.8565317, 77

In [14]:
Lat=[]
lon=[]
for i in coords:
    Lat.append(i[0])
    lon.append(i[1])

In [15]:
coordinates=pd.DataFrame({"Latitude": Lat,"Longitude":lon})
coordinates.head()

,Latitude,Longitude
0,15.8877,75.7047
1,13.0142,77.6519
2,13.8346,75.7339
3,12.9938,77.5391
4,12.9358,77.6668


In [16]:
banglore_df['latitude']=coordinates['Latitude']
banglore_df['longitude']=coordinates['Longitude']
banglore_df.head()

,Neighborhood,latitude,longitude
0,Banashankari,15.8877,75.7047
1,Banaswadi,13.0142,77.6519
2,Basavanagudi,13.8346,75.7339
3,Basaveshwaranagar,12.9938,77.5391
4,Bellandur,12.9358,77.6668


In [17]:
banglore_df=banglore_df[banglore_df.latitude!='n'].reset_index(drop=True)
banglore_df

,Neighborhood,latitude,longitude
0,Banashankari,15.8877,75.7047
1,Banaswadi,13.0142,77.6519
2,Basavanagudi,13.8346,75.7339
3,Basaveshwaranagar,12.9938,77.5391
4,Bellandur,12.9358,77.6668
5,Bengaluru Pete,12.9997,76.0933
6,Bharathnagar,12.9786,77.4779
7,Bilekahalli,12.8996,77.6103
8,Bommanahalli,12.9089,77.6239
9,Bommasandra,12.8162,77.6916


In [18]:
address='Banglore, India'
geolocator=Nominatim(user_agent='My-application')
location=geolocator.geocode(address)
latitude=location.latitude
longitude=location.longitude
print('The geographical coordinates of Banglore are {},{}'.format(latitude,longitude))

The geographical coordinates of Banglore are 18.3903609,78.8087788


In [19]:
map_banglore=folium.Map(location=[latitude,longitude],zoom_start=10)

for lat,lon,neigh in zip(banglore_df['latitude'],banglore_df['longitude'],banglore_df['Neighborhood']):
    label='{}'.format(neigh)
    label=folium.Popup(label,parse_html=True)
    folium.CircleMarker([lat,lon],
                        radius=5,
                        popup=label,
                        color='blue',
                        fill=True,
                        fill_color='#3186cc',
                        fill_opacity='0.7',
                        parse_html=False).add_to(map_banglore)
map_banglore

<h3>Use the Four Square api to get the geographical coordinates

In [20]:
CLIENT_ID = 'SSCE0YASRAL32EWIFLK2SSHEG3MXZ1TNBX0Y5EVJA2K5SMCD' # your Foursquare ID
CLIENT_SECRET = 'D554AL4S5ZHW5YQCIG4XMDVOJOPMH2HCU0EKCCRXNCX12EHZ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: SSCE0YASRAL32EWIFLK2SSHEG3MXZ1TNBX0Y5EVJA2K5SMCD
CLIENT_SECRET:D554AL4S5ZHW5YQCIG4XMDVOJOPMH2HCU0EKCCRXNCX12EHZ


<h5>Retrive The top 100 venues in between 500 metres radius

In [25]:
radius = 500
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(banglore_df['latitude'], banglore_df['longitude'], banglore_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [26]:
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df

(1007, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Banaswadi,13.014162,77.651854,Tamarind,13.012676,77.647711,Indian Restaurant
1,Banaswadi,13.014162,77.651854,Saffron,13.012763,77.647837,Vegetarian / Vegan Restaurant
2,Banaswadi,13.014162,77.651854,Kanti Sweets,13.012989,77.649907,Indian Restaurant
3,Banaswadi,13.014162,77.651854,De Grand,13.016421,77.650597,BBQ Joint
4,Banaswadi,13.014162,77.651854,Foodworld Super Market,13.013297,77.648234,Convenience Store
5,Banaswadi,13.014162,77.651854,OvenTreats,13.014426,77.647719,Bakery
6,Banaswadi,13.014162,77.651854,Idli-Yo-Idli,13.014845,77.656311,Vegetarian / Vegan Restaurant
7,Basaveshwaranagar,12.993757,77.539132,Polar Bear,12.991461,77.538832,Ice Cream Shop
8,Basaveshwaranagar,12.993757,77.539132,Priyadarshini Grand,12.990235,77.538020,Fast Food Restaurant
9,Basaveshwaranagar,12.993757,77.539132,Gold's gym,12.991855,77.538492,Gym


In [27]:
venues_df.groupby(['Neighborhood']).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Adugodi,6,6,6,6,6,6
Austin Town,3,3,3,3,3,3
BTM Layout,37,37,37,37,37,37
"Bagalur, Bangalore Urban",1,1,1,1,1,1
Banaswadi,14,14,14,14,14,14
Basaveshwaranagar,40,40,40,40,40,40
Bengaluru Pete,2,2,2,2,2,2
Bilekahalli,1,1,1,1,1,1
Bommanahalli,5,5,5,5,5,5


In [29]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 169 uniques categories.


In [30]:
venues_df['VenueCategory'].unique()[:50]

array(['Indian Restaurant', 'Vegetarian / Vegan Restaurant', 'BBQ Joint',
       'Convenience Store', 'Bakery', 'Ice Cream Shop',
       'Fast Food Restaurant', 'Gym', 'Liquor Store', 'Café',
       'Pizza Place', 'Bus Station', 'Department Store',
       'Mobile Phone Shop', 'Sporting Goods Shop', 'Miscellaneous Shop',
       'Food Truck', 'Park', 'Tea Room', 'Mountain', 'ATM', 'Pharmacy',
       'Auto Garage', 'Business Service', 'Home Service', 'Restaurant',
       'Chinese Restaurant', 'Dim Sum Restaurant', 'Sandwich Place',
       'Coffee Shop', 'Snack Place', 'Italian Restaurant', 'Diner',
       'Kerala Restaurant', 'Flower Shop', 'Gourmet Shop', 'Supermarket',
       'Train Station', 'Indian Chinese Restaurant',
       'South Indian Restaurant', 'Middle Eastern Restaurant',
       'Shopping Mall', 'Hotel', 'Pub', 'Kebab Restaurant',
       'Grocery Store', 'Deli / Bodega', 'Sports Bar',
       'Rajasthani Restaurant', 'Vietnamese Restaurant'], dtype=object)

In [33]:
banglore_onehot=pd.get_dummies(venues_df[['VenueCategory']],prefix="",prefix_sep="")
banglore_onehot['Neighborhood']=venues_df['Neighborhood']
fixed_columns = [banglore_onehot.columns[-1]] + list(banglore_onehot.columns[:-1])
banglore_onehot=banglore_onehot[fixed_columns]
print(banglore_onehot.shape)
banglore_onehot.head()

(1007, 170)


,Neighborhood,ATM,Accessories Store,American Restaurant,Andhra Restaurant,Arcade,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Badminton Court,Bakery,Bar,Bed & Breakfast,Bengali Restaurant,Bistro,Boarding House,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Burger Joint,Bus Line,Bus Station,Bus Stop,Business Service,Café,Camera Store,Candy Store,Chaat Place,Chettinad Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,Convenience Store,Cosmetics Shop,Creperie,Cricket Ground,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Donut Shop,Dry Cleaner,Eastern European Restaurant,Electronics Store,English Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Furniture / Home Store,Gaming Cafe,Gastropub,General Entertainment,Golf Course,Gourmet Shop,Grocery Store,Gun Shop,Gym,Gym / Fitness Center,Hardware Store,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Hyderabadi Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indonesian Meatball Place,Indonesian Restaurant,Inn,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karnataka Restaurant,Kebab Restaurant,Kerala Restaurant,Korean Restaurant,Lake,Light Rail Station,Liquor Store,Lounge,Market,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Mughlai Restaurant,Multiplex,Museum,Music Store,Music Venue,Nightclub,Noodle House,North Indian Restaurant,Office,Optical Shop,Paintball Field,Paper / Office Supplies Store,Park,Performing Arts Venue,Pharmacy,Pizza Place,Platform,Playground,Plaza,Pool,Pub,Punjabi Restaurant,Rajasthani Restaurant,Restaurant,Road,Sandwich Place,Science Museum,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Smoke Shop,Snack Place,Soccer Field,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Sporting Goods Shop,Sports Bar,Squash Court,Steakhouse,Sundanese Restaurant,Supermarket,Sushi Restaurant,Tea Room,Tex-Mex Restaurant,Track Stadium,Train Station,Udupi Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Women's Store
0,Banaswadi,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Banaswadi,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,Banaswadi,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Banaswadi,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Banaswadi,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [35]:
banglore_grouped = banglore_onehot.groupby(["Neighborhood"]).mean().reset_index()

print(banglore_grouped.shape)
banglore_grouped

(95, 170)


,Neighborhood,ATM,Accessories Store,American Restaurant,Andhra Restaurant,Arcade,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Badminton Court,Bakery,Bar,Bed & Breakfast,Bengali Restaurant,Bistro,Boarding House,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Burger Joint,Bus Line,Bus Station,Bus Stop,Business Service,Café,Camera Store,Candy Store,Chaat Place,Chettinad Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,Convenience Store,Cosmetics Shop,Creperie,Cricket Ground,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Donut Shop,Dry Cleaner,Eastern European Restaurant,Electronics Store,English Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Furniture / Home Store,Gaming Cafe,Gastropub,General Entertainment,Golf Course,Gourmet Shop,Grocery Store,Gun Shop,Gym,Gym / Fitness Center,Hardware Store,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Hyderabadi Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indonesian Meatball Place,Indonesian Restaurant,Inn,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karnataka Restaurant,Kebab Restaurant,Kerala Restaurant,Korean Restaurant,Lake,Light Rail Station,Liquor Store,Lounge,Market,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Mughlai Restaurant,Multiplex,Museum,Music Store,Music Venue,Nightclub,Noodle House,North Indian Restaurant,Office,Optical Shop,Paintball Field,Paper / Office Supplies Store,Park,Performing Arts Venue,Pharmacy,Pizza Place,Platform,Playground,Plaza,Pool,Pub,Punjabi Restaurant,Rajasthani Restaurant,Restaurant,Road,Sandwich Place,Science Museum,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Smoke Shop,Snack Place,Soccer Field,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Sporting Goods Shop,Sports Bar,Squash Court,Steakhouse,Sundanese Restaurant,Supermarket,Sushi Restaurant,Tea Room,Tex-Mex Restaurant,Track Stadium,Train Station,Udupi Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Women's Store
0,Adugodi,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.166667,0.0,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,0.000000,0.0,0.000000,0.000000,0.000000,0.166667,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Austin Town,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.333333,0.00

In [36]:
len(banglore_grouped[banglore_grouped["Shopping Mall"] > 0])

6

In [38]:
banglore_mall=banglore_grouped[['Neighborhood','Shopping Mall']]
banglore_mall.head()

,Neighborhood,Shopping Mall
0,Adugodi,0.0
1,Austin Town,0.0
2,BTM Layout,0.0
3,"Bagalur, Bangalore Urban",0.0
4,Banaswadi,0.0


<h1>Cluster Neighborhoods

<b>Run k-means to cluster the neighborhoods in Kuala Lumpur into 3 clusters.

In [ ]:
b_clusters=3
b_clustering=banglore_mall.drop(["Neighborhood"], 1)
kmeans=KMeans(n_clusters=b_clusters,random_state=0).fit(b_clustering)
kmeans.labels_[0:10]

In [42]:
ban_merged=banglore_mall.copy()
ban_merged['Cluster Labels']=kmeans.labels_

In [46]:
ban_merged

,Neighborhood,Shopping Mall,Cluster Labels,latitude,longitude
0,Adugodi,0.000000,0,12.9428,77.6104
1,Austin Town,0.000000,0,12.9613,77.6153
2,BTM Layout,0.000000,0,12.9152,77.6103
3,"Bagalur, Bangalore Urban",0.000000,0,13.1332,77.6687
4,Banaswadi,0.000000,0,13.0142,77.6519
4,Banaswadi,0.000000,0,13.0142,77.6519
5,Basaveshwaranagar,0.000000,0,12.9938,77.5391
5,Basaveshwaranagar,0.000000,0,12.9938,77.5391
6,Bengaluru Pete,0.000000,0,12.9997,76.0933
7,Bilekahalli,0.000000,0,12.8996,77.6103


In [ ]:
ban_merged = ban_merged.join(banglore_df.set_index("Neighborhood"), on="Neighborhood")

In [47]:
ban_merged.head()

,Neighborhood,Shopping Mall,Cluster Labels,latitude,longitude
0,Adugodi,0.0,0,12.9428,77.6104
1,Austin Town,0.0,0,12.9613,77.6153
2,BTM Layout,0.0,0,12.9152,77.6103
3,"Bagalur, Bangalore Urban",0.0,0,13.1332,77.6687
4,Banaswadi,0.0,0,13.0142,77.6519


In [48]:
ban_merged.sort_values(['Cluster Labels'],inplace=True)
ban_merged

,Neighborhood,Shopping Mall,Cluster Labels,latitude,longitude
0,Adugodi,0.000000,0,12.9428,77.6104
66,Padmanabhanagar,0.000000,0,12.9181,77.5577
65,Nayandahalli,0.000000,0,12.9413,77.5212
64,Nandini Layout,0.000000,0,13.0104,77.5378
63,Murugeshpalya,0.000000,0,12.9589,77.6556
62,"Murphy Town, Bangalore",0.000000,0,12.9802,77.6274
61,Mathikere,0.000000,0,13.0329,77.5574
60,Marathahalli,0.000000,0,12.9553,77.6984
59,Mahalakshmi Layout,0.000000,0,13.0113,77.5447
57,Madiwala,0.000000,0,12.9212,77.618


<b>Finally, let's visualize the resulting clusters

In [50]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(b_clusters)
ys = [i+x+(i*x)**2 for i in range(b_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ban_merged['latitude'], ban_merged['longitude'], ban_merged['Neighborhood'], ban_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<h1>Examine Clusters

In [54]:
ban_merged.loc[ban_merged['Cluster Labels']==0]

,Neighborhood,Shopping Mall,Cluster Labels,latitude,longitude
0,Adugodi,0.000000,0,12.9428,77.6104
66,Padmanabhanagar,0.000000,0,12.9181,77.5577
65,Nayandahalli,0.000000,0,12.9413,77.5212
64,Nandini Layout,0.000000,0,13.0104,77.5378
63,Murugeshpalya,0.000000,0,12.9589,77.6556
62,"Murphy Town, Bangalore",0.000000,0,12.9802,77.6274
61,Mathikere,0.000000,0,13.0329,77.5574
60,Marathahalli,0.000000,0,12.9553,77.6984
59,Mahalakshmi Layout,0.000000,0,13.0113,77.5447
57,Madiwala,0.000000,0,12.9212,77.618


In [51]:
ban_merged.loc[ban_merged['Cluster Labels']==1]

,Neighborhood,Shopping Mall,Cluster Labels,latitude,longitude
82,Taverekere,0.039216,1,12.9329,77.6084
58,"Mahadevapura, Bangalore",0.062500,1,12.9935,77.6924
78,Seshadripuram,0.041667,1,12.9932,77.5753
94,Yeswanthpur,0.052632,1,13.0238,77.5529


In [53]:
ban_merged.loc[ban_merged['Cluster Labels']==2]

,Neighborhood,Shopping Mall,Cluster Labels,latitude,longitude
17,Cooke Town,0.111111,2,13.0028,77.6247


<b>Observations

In cluster2 areas has intense competition of Shopping malls whereas ,cluster1 has moderate competition of shopping malls and cluster 0 has no competition.So, it is advisible to the Investors or Developers to install the shopping malls in either cluster 0 or 1 areas which has no or less competition.